# Electric Vehicle Growth Prediction — Data Cleaning & Preparation

This notebook creates a clean, area-level dataset from VIN-level EV registrations.
It performs: loading, column normalization, missing-value handling, coordinate parsing,
aggregation by 2020 Census tract, target creation, and export for modeling.

## Step 0: Imports & Settings

In [5]:
import pandas as pd
import numpy as np
import os, re
from pathlib import Path

if Path.cwd().name == "notebooks":
    os.chdir(Path.cwd().parent)
print("Working directory set to:", Path.cwd())

pd.set_option("display.max_columns", 50)
print("Pandas version:", pd.__version__)

Working directory set to: C:\Users\Agree\Desktop\electric-vehicle-growth-prediction
Pandas version: 2.2.3


## Step 1: Load the raw dataset

In [6]:
# Tries 'data/' first; falls back to current directory
candidate_paths = [
    "data/Electric_Vehicle_Population_Data.csv",
    "Electric_Vehicle_Population_Data.csv"
]

for p in candidate_paths:
    if os.path.exists(p):
        raw_path = p
        break
else:
    raise FileNotFoundError("Could not find Electric_Vehicle_Population_Data.csv")

df = pd.read_csv(raw_path)
print("Loaded shape:", df.shape)
df.head()

Loaded shape: (264628, 17)


,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,WA1E2AFY8R,Thurston,Olympia,WA,98512.0,2024,AUDI,Q5 E,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,23.0,0.0,22.0,263239938,POINT (-122.90787 46.9461),PUGET SOUND ENERGY INC,5.306701e+10
1,WAUUPBFF4J,Yakima,Wapato,WA,98951.0,2018,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16.0,0.0,15.0,318160860,POINT (-120.42083 46.44779),PACIFICORP,5.307794e+10
2,1N4AZ0CP0F,King,Seattle,WA,98125.0,2015,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,84.0,0.0,46.0,184963586,POINT (-122.30253 47.72656),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
3,WA1VAAGE5K,King,Kent,WA,98031.0,2019,AUDI,E-TRON,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,204.0,0.0,11.0,259426821,POINT (-122.17743 47.41185),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
4,7SAXCAE57N,Snohomish,Bothell,WA,98021.0,2022,TESLA,MODEL X,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,1.0,208182236,POINT (-122.18384 47.8031),PUGET SOUND ENERGY INC,5.306105e+10


## Step 2: Normalize column names

In [7]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace("\n", "_")
)
df.columns.tolist()

['vin_(1-10)',
 'county',
 'city',
 'state',
 'postal_code',
 'model_year',
 'make',
 'model',
 'electric_vehicle_type',
 'clean_alternative_fuel_vehicle_(cafv)_eligibility',
 'electric_range',
 'base_msrp',
 'legislative_district',
 'dol_vehicle_id',
 'vehicle_location',
 'electric_utility',
 '2020_census_tract']

## Step 3: Quick missing-value audit

In [8]:
na_report = df.isnull().sum().sort_values(ascending=False)
na_report

legislative_district                                 659
vehicle_location                                      17
2020_census_tract                                      9
city                                                   9
electric_utility                                       9
postal_code                                            9
county                                                 9
electric_range                                         4
base_msrp                                              4
dol_vehicle_id                                         0
vin_(1-10)                                             0
clean_alternative_fuel_vehicle_(cafv)_eligibility      0
model                                                  0
make                                                   0
model_year                                             0
state                                                  0
electric_vehicle_type                                  0
dtype: int64

## Step 4: Type casting & missing-value handling

In [9]:
# Cast numerics (coerce errors to NaN so we can handle them)
for col in ["model_year", "electric_range", "base_msrp"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Drop rows with critical missing identifiers (these are rare)
critical_cols = ["model_year", "make", "electric_vehicle_type", "2020_census_tract"]
present_critical = [c for c in critical_cols if c in df.columns]
df = df.dropna(subset=present_critical)

# Median-impute numeric columns commonly missing
for col in ["electric_range", "base_msrp"]:
    if col in df.columns:
        med = df[col].median()
        df[col] = df[col].fillna(med)

# If vehicle_location exists, keep only rows with coordinates (they're needed for mapping)
if "vehicle_location" in df.columns:
    df = df.dropna(subset=["vehicle_location"])
print("Post-cleaning shape:", df.shape)
df.head()

Post-cleaning shape: (264611, 17)


,vin_(1-10),county,city,state,postal_code,model_year,make,model,electric_vehicle_type,clean_alternative_fuel_vehicle_(cafv)_eligibility,electric_range,base_msrp,legislative_district,dol_vehicle_id,vehicle_location,electric_utility,2020_census_tract
0,WA1E2AFY8R,Thurston,Olympia,WA,98512.0,2024,AUDI,Q5 E,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,23.0,0.0,22.0,263239938,POINT (-122.90787 46.9461),PUGET SOUND ENERGY INC,5.306701e+10
1,WAUUPBFF4J,Yakima,Wapato,WA,98951.0,2018,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16.0,0.0,15.0,318160860,POINT (-120.42083 46.44779),PACIFICORP,5.307794e+10
2,1N4AZ0CP0F,King,Seattle,WA,98125.0,2015,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,84.0,0.0,46.0,184963586,POINT (-122.30253 47.72656),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
3,WA1VAAGE5K,King,Kent,WA,98031.0,2019,AUDI,E-TRON,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,204.0,0.0,11.0,259426821,POINT (-122.17743 47.41185),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
4,7SAXCAE57N,Snohomish,Bothell,WA,98021.0,2022,TESLA,MODEL X,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0.0,0.0,1.0,208182236,POINT (-122.18384 47.8031),PUGET SOUND ENERGY INC,5.306105e+10


## Step 5: Parse `vehicle_location` (POINT long lat) to `longitude` and `latitude`

In [10]:
if "vehicle_location" in df.columns:
    # Expected format: 'POINT (LONG LAT)'
    lon = df["vehicle_location"].str.extract(r"POINT\s*\(([-\d\.]+)")[0]
    lat = df["vehicle_location"].str.extract(r"POINT\s*\([-\d\.]+\s+([\-\d\.]+)\)")[0]
    df["longitude"] = pd.to_numeric(lon, errors="coerce")
    df["latitude"]  = pd.to_numeric(lat, errors="coerce")
    # drop rows still missing parsed coords
    df = df.dropna(subset=["longitude", "latitude"])
df[["vehicle_location","longitude","latitude"]].head() if "vehicle_location" in df.columns else df.head()

,vehicle_location,longitude,latitude
0,POINT (-122.90787 46.9461),-122.90787,46.94610
1,POINT (-120.42083 46.44779),-120.42083,46.44779
2,POINT (-122.30253 47.72656),-122.30253,47.72656
3,POINT (-122.17743 47.41185),-122.17743,47.41185
4,POINT (-122.18384 47.8031),-122.18384,47.80310


## Step 6: Aggregate to area-level (2020 Census tract)

In [11]:
if "2020_census_tract" not in df.columns:
    raise KeyError("Missing '2020_census_tract' column needed for aggregation.")

# Compute percent BEV robustly
percent_bev_fn = lambda x: x.astype(str).str.contains("BEV", case=False, na=False).mean()

area_df = df.groupby("2020_census_tract").agg(
    vehicle_count=("vin_(1-10)", "count"),
    electric_range=("electric_range", "mean"),
    base_msrp=("base_msrp", "mean"),
    model_year=("model_year", "mean"),
    percent_bev=("electric_vehicle_type", percent_bev_fn)
).reset_index()

print("Area-level shape:", area_df.shape)
area_df.head()

Area-level shape: (2328, 6)


,2020_census_tract,vehicle_count,electric_range,base_msrp,model_year,percent_bev
0,1.001020e+09,2,19.5,0.0,2022.0,0.5
1,1.001021e+09,1,25.0,0.0,2022.0,0.0
2,1.045020e+09,1,0.0,0.0,2023.0,1.0
3,1.081042e+09,1,220.0,0.0,2019.0,1.0
4,1.117031e+09,1,259.0,0.0,2020.0,1.0


## Step 7: Create classification target `high_adoption`

In [12]:
median_count = area_df["vehicle_count"].median()
area_df["high_adoption"] = (area_df["vehicle_count"] > median_count).astype(int)
area_df["high_adoption"].value_counts()

high_adoption
0    1168
1    1160
Name: count, dtype: int64

## Step 8: Export cleaned area-level dataset

In [13]:
os.makedirs("data", exist_ok=True)
out_csv = "data/cleaned_ev_area_data.csv"
area_df.to_csv(out_csv, index=False)
print("Saved to:", out_csv)

Saved to: data/cleaned_ev_area_data.csv
